# An Introduction to Hyperspectral Imaging with CUVIS SDK

In [ ]:
import os # File system operation
import numpy as np # Load numeric data operations
import cuvis # Load the Cubert SDK
import traceback # Debugging errors
import typing # Typehints for python data
from IPython.display import display, clear_output # Interactive tools for Notebooks
from tqdm import tqdm # Interactive progress bar
import ipywidgets as widgets
import traitlets
from ipyfilechooser import FileChooser
import cv2 # OpenCV proxy library
from sklearn.decomposition import PCA # Data dimensionality reduction
import matplotlib.pyplot as plt # Visualization of images
import cuvis.classificator as cuvClass # Beta Classification Engine

### Select and Image to Load

In [ ]:
lib_dir = os.getenv("CUVIS")
default_path = os.path.normpath(os.path.join(lib_dir, os.path.pardir, "sdk", "sample_data", "set1"))
fc = FileChooser(default_path)
display(fc)

### Load the sample data into memory

In [ ]:
### Create a Measurement from the Image
try:
    mesu = cuvis.Measurement(fc.selected)
except Exception as e:
    print(traceback.print_exc())

### Explore the Metadata on the file

In [ ]:
def prettyprint_attributes(mesu: cuvis.Measurement) -> None:
    for attribute in dir(mesu):
        if type(getattr(mesu, attribute)).__name__ not in ['builtin_function_or_method', 'method'] and '__' not in attribute:
            print(f'{attribute}:\t{getattr(mesu, attribute)}')

# Let's examine our measurement
prettyprint_attributes(mesu)

### Now let's look at the actual data array

In [ ]:
data = mesu.Data["cube"].array
# Let's look at the shape of the hypercube
print(f'Rows: {data.shape[0]}, Columns: {data.shape[1]}, Bands: {data.shape[2]}')

Hyperspectral data from Cubert is read into a 3 dimensional structure. Each images, has a width, height, and band dimension. You can access specific points of this data by using numeric python operations using the popular `numpy` library. If our hypercube is called `data` we can perform some operations to find specific portions
```
data[:,:,0] # Select the first band in the cube
data[0,0,:] # Select the spectral profile for a specific pixel
data[10:20,10:20,:] # Select a square region of interest (ROI) using array slicing
```
To read more about all the things you can do with array silicing to select, and downsample datacubes, please read the [official numpy documentation](https://numpy.org/doc/stable/user/basics.indexing.html).

Below let's extract the first wavelength channel and plot it.

In [ ]:
plt.imshow(data[:,:,0])
plt.show()

### Visualizaiton with IPyWidgets

Jupyter Notebooks are also great for building interactive tools for data analysis. In the cell below, we have constructed a simple graphical user interface to explore hyperspectral data. This leverages the [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/) library.

In [ ]:
%matplotlib inline
out = widgets.Output()
out_spectra = widgets.Output()
slider = widgets.SelectionSlider(
    options=mesu.Data["cube"].wavelength,
    value=mesu.Data["cube"].wavelength[0],
    description='Channel:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
x_val = widgets.BoundedIntText(
    value=0,
    min=0,
    max=data.shape[0]-1,
    step=1,
    description='X Coord:',
    disabled=False
)
y_val = widgets.BoundedIntText(
    value=0,
    min=0,
    max=data.shape[1]-1,
    step=1,
    description='Y Coord:',
    disabled=False
)
neighborhood = widgets.BoundedIntText(
    value=3,
    min=1,
    max=100,
    step=1,
    description='Avg. Radius',
    disabled=False
)   
# Create initial bytes
def on_value_change(change):
    # Update the visualization
    if 'Channel' in change['owner'].description and isinstance(change['new'], int) and change['new'] in mesu.Data["cube"].wavelength:
        new_channel = change['new']
        show_channel(new_channel)
    elif 'X Coord' in change['owner'].description and isinstance(change['new'], int):
        plot_spectra()
        show_channel(slider.value)
    elif 'Y Coord' in change['owner'].description and isinstance(change['new'], int):
        plot_spectra()
        show_channel(slider.value)
    elif 'Avg. Radius' in change['owner'].description and isinstance(change['new'], int):
        plot_spectra()
        show_channel(slider.value)

def show_channel(new_channel: int) -> None:
    x = x_val.value
    y = y_val.value
    with out:
        clear_output(True)
        new_channel = mesu.Data["cube"].wavelength.index(new_channel)
        plt.imshow(data[:,:,new_channel], cmap='gray', extent=[0, data.shape[1], 0, data.shape[0]])
        plt.plot([x], [y], marker="o", markersize=neighborhood.value+2, markeredgecolor="red", markerfacecolor="green")
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()
        
def average_spectra(radius: int) -> np.array:
    x = x_val.value
    y = y_val.value
    min_x = x-radius if x-radius > 0 else 0
    min_y = y-radius if y-radius > 0 else 0
    sample = data[min_x:x+radius,min_y:y+radius,:]
    sample = sample.reshape(sample.shape[0]*sample.shape[1],sample.shape[2])
    return np.mean(sample,axis=0)
            
def plot_spectra() -> None:
    x = x_val.value
    y = y_val.value
    with out_spectra:
        clear_output(True)
        sample = average_spectra(neighborhood.value)
        plt.plot(mesu.Data["cube"].wavelength, sample)
        plt.title(f'Spectra of Point: {x}, {y}')
        plt.xlabel('Wavelength (nm)')
        plt.ylabel('Counts')
        plt.grid(True)
        plt.show()
            
# Show the inital image
slider.observe(on_value_change)
x_val.observe(on_value_change)
y_val.observe(on_value_change)
neighborhood.observe(on_value_change)
display(
    widgets.VBox([
        widgets.HBox([
            out,
            out_spectra
        ]),
        widgets.HBox([
            slider,
            x_val,
            y_val,
            neighborhood
        ])
])
)
show_channel(slider.value)
plot_spectra()

This graphical user interface gives us a simple user interface to change through the image channels and highlight spectral of samples. Dragging the slider will update the selected image channel. The plot on the right shows the uncorrected spectral profile for the pixels contained in the green circle on the left. Try playing with the UI to explore some of your data!

### Color Space Representation

We can also use the same GUI tools to visual a color representation of the image.

In [ ]:
red = widgets.SelectionSlider(
    options=mesu.Data["cube"].wavelength,
    value=mesu.Data["cube"].wavelength[0],
    description='Ch 1 (R):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
green = widgets.SelectionSlider(
    options=mesu.Data["cube"].wavelength,
    value=mesu.Data["cube"].wavelength[1],
    description='Ch 2 (G):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
blue = widgets.SelectionSlider(
    options=mesu.Data["cube"].wavelength,
    value=mesu.Data["cube"].wavelength[2],
    description='Ch 3 (B):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
common_vis = widgets.RadioButtons(
    options=['Off', 'RGB', 'CIR'],
    description='Common Visualizations:',
    disabled=False
)
def create_rgb(_: traitlets.utils.bunch.Bunch) -> None:
    red_channel = data[:,:,mesu.Data["cube"].wavelength.index(red.value)]
    blue_channel = data[:,:,mesu.Data["cube"].wavelength.index(blue.value)]
    green_channel = data[:,:,mesu.Data["cube"].wavelength.index(green.value)]
    with out_rgb:
        clear_output(True)
        rgb = np.dstack((red_channel, green_channel, blue_channel))
        rgb_norm = cv2.normalize(rgb, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        plt.imshow(rgb_norm, extent=[0, data.shape[1], 0, data.shape[0]])
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()

def set_bands(event: traitlets.utils.bunch.Bunch) -> None:
    if event.get('name') == 'value':
        # Set the slider values
        if event.get('new') == 'CIR':
            red.value = 842
            green.value = 682
            blue.value = 562
        elif event.get('new') == 'RGB':
            red.value = 682
            green.value = 562
            blue.value = 498
        
# Show the inital image
out_rgb = widgets.Output()
red.observe(create_rgb)
blue.observe(create_rgb)
green.observe(create_rgb)
common_vis.observe(set_bands)
display(
    widgets.VBox([
        widgets.HBox([
            out_rgb,
            common_vis
        ]),
        widgets.HBox([
            red,
            green,
            blue,
        ])
])
)
create_rgb(None)

### Reflectance Normalization and Clustering

Hyperspectral sensors have a wide dynamic range depending on the integraiton time and the amount of available illumination. For most classification algorithms, it is useful if we can have our values scaled between 0 and 1. To achieve this, we apply reflectance calibraiton, which takes our data and compares it to the absolute maximum and minimum possible signals given the current conditions. Behind the scenes, the Cuvis SDK applies the following equation to the datacube.

$$\text{signal}_{\text{cal}} = \frac{\text{signal}_{\text{raw}}-\text{dark}}{\text{white}-\text{dark}}$$


#### Load white reflectance cube

In [ ]:
fc = FileChooser(fc.selected_path) # Note, this should be in the white_000 directory
display(fc)

In [ ]:
white_ref = cuvis.Measurement(fc.selected)

#### Load dark reference

In [ ]:
fc = FileChooser(fc.selected_path) # dark_000 folder
display(fc)

In [ ]:
dark_ref = cuvis.Measurement(fc.selected)

### Load distance reference

In [ ]:
fc = FileChooser(fc.selected_path) # Calibration folder
display(fc)

In [ ]:
distance = cuvis.Measurement(fc.selected)

In [ ]:
from copy import deepcopy
factoryDir = 'C:/Program Files/Cuvis/sdk/sample_data/set1/factory' # Sensor factory calibration directory
calibration = cuvis.Calibration(calibdir=factoryDir) # Create the calibration directory
processingContext = cuvis.ProcessingContext(calibration) # Create the processing context given the calibration
processingContext.setReference(dark_ref, "Dark") # Load the dark reference
processingContext.setReference(white_ref, "White") # Load the white reference
processingContext.setReference(distance, "Distance")

modes = ["Raw",
         "DarkSubtract",
         "Reflectance",
         ]
procArgs = cuvis.CubertProcessingArgs()
outputCubes = {}
for mode in modes:
    procArgs.ProcessingMode = mode
    if processingContext.isCapable(mesu, procArgs):
        print("processing to mode {}...".format(mode))
        processingContext.setProcessingArgs(procArgs)
        outputCubes[mode] = deepcopy(processingContext.apply(mesu))
    else:
        print("Cannot process to {} mode!".format(mode))

Now that our cube is normalized, let's view a histogram of the points. Moreover, let's compare a classical calculation to Cuvis' results.

In [ ]:
outputCubes['Reflectance'].Data['cube'].array
counts, bins = np.histogram(outputCubes['Reflectance'].Data['cube'].array.flatten())
plt.stairs(counts, bins)
plt.xlabel('Value')
plt.ylabel('Counts')
plt.title('Cuvis Calculated Reflectance')
plt.grid(True)
plt.show()

In [ ]:
output_cube = (data-dark_ref.Data['cube'].array) / (white_ref.Data['cube'].array-dark_ref.Data['cube'].array)
counts, bins = np.histogram(output_cube.flatten())
plt.stairs(counts, bins)
plt.xlabel('Value')
plt.ylabel('Counts')
plt.title('Manually Calculated Reflectance')
plt.grid(True)
plt.show()

### PCA and explained total variance

Hyperspectral data is often considered "big data", which makes it a challenging domain for real-time data processing pipelines. One thing that often helps simplify data analysis is to perform a dimensionality reduction technique to reduce the number of spectral components used to represent the spectrum. Here we will use Principal Components Analysis.

In [ ]:
pca = PCA()
pca.fit(output_cube.reshape(-1, output_cube.shape[-1]))

PCA applies a linear reprojection to the original feature space, which allows an interpretation of the components by the amount of variation explained in the data. Below, we will plot the total explained variance as a function of the number of components in our dataset

In [ ]:
plt.plot(range(output_cube.shape[2]),np.cumsum(pca.explained_variance_ratio_)*100)
plt.xlabel('Number of Components (Spectral Bands)')
plt.ylabel('Explained Variance (%)')
plt.title('Principal Component Analysis on HyperCube')
plt.grid()
plt.show()

As you can clearly see from the results above, <10 components should be needed to explain almost the entirety of the variation in the dataset. Other dimensionality techniques, like Singular Value Decomposition, Minimum Noise Fraction, Factor Analysis, and Nonnegative Matrix Factorization can be subsitituted here, and make for some thrilling mathematical reading.

In [ ]:
output_cube_reduced = pca.transform(output_cube.reshape(-1, output_cube.shape[-1]))
reduced_cube = output_cube_reduced.reshape(output_cube.shape)

In [ ]:
components_to_keep = 10 # Pick the total number of components to use
reduced_cube_final = reduced_cube[:,:,:components_to_keep]

### K-Means Clustering

As a practical demonstration of the CUVIS processing pipeline, we will now apply an _unsupervised_ machine learning algorithm, K-Means clustering, to our PCA transformed datacube.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(
    init="random",
    n_clusters=4, # This value is up to you! How many classes do you think should be in the image?
    n_init=10,
    max_iter=300,
    random_state=42
)
flattened_data = reduced_cube_final.reshape(-1, reduced_cube_final.shape[-1])
kmeans.fit(flattened_data)
preds = kmeans.predict(flattened_data)
plt.imshow(preds.reshape((reduced_cube.shape[0],reduced_cube.shape[1])))
plt.title('PCA K-Means Prediction')
plt.show()

Now let's compare this to the results if we use the entire data cube without dimensionality reduction.

In [ ]:
kmeans = KMeans(
    init="random",
    n_clusters=4, # This value is up to you! How many classes do you think should be in the image?
    n_init=10,
    max_iter=300,
    random_state=42
)
flattened_data = output_cube.reshape(-1, output_cube.shape[-1])
kmeans.fit(flattened_data)
preds = kmeans.predict(flattened_data)
plt.imshow(preds.reshape((output_cube.shape[0], output_cube.shape[1])))
plt.title('Full Data K-Means Prediction')
plt.show()

Note the colors in the image might change between images, but the shape contours should converge to the same result consistently. As you can see from the results, a reduced number of components is sufficient to differentiate the plant from the backgorund. Try rerunning these calculations with a different number of clusters, and then compare the results!